# Audio Recognition Using Siamese Network

In the last tutorial, we saw how to use the siamese networks to recognize a face. Now we will see how to use the siamese networks to recognize the audio. We will train our network to differentiate between a dog's and a cat's sound. The dataset of cats and dogs audio can be downloaded from here https://www.kaggle.com/mmoreaux/audio-cats-and-dogs#cats_dogs.zip

Once we have downloaded the data, we fragment our data into three folders Dogs, Sub_dogs, and Cats. In Dogs and Sub_dogs, we place the dog's barking audio and in Cats folder, we place the cat's audio. The objective of our network is to recognize whether the audio is the dog's barking sound or some different sound. As we know for a Siamese network, we need to feed input as a pair, we select an audio from Dogs and Sub_dogs folder and mark it as a genuine pair and we select an audio from Dogs and Cats folder and mark it as an imposite pair. That is, (dogs, subdogs) is genuine pair and (dogs, cats) is imposite pair.

Now we will step by step how to train our siamese network to recognize whether the audio is the dog's barking sound or some different sound. First, We will load all the necessary libraries:

In [1]:

# In[1]:
from preprocessing import cv_preprocessing

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
print("sad imports")

#basic imports
import glob
#import IPython
from random import randint
print("sad imports")

#data processing
import numpy as np
print("sad imports")
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, auc, average_precision_score, precision_recall_curve
import matplotlib.pyplot as plt

#modelling
from sklearn.model_selection import train_test_split
print("sad imports")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras import backend as K
print("sad imports")

from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model
print("sad imports")

from keras.optimizers import RMSprop
from load_data import load_data


Instructions for updating:
non-resource variables are not supported in the long term
sad imports
sad imports
sad imports
sad imports
sad imports
sad imports


In [2]:

def print_results(y_true, y_pred, name=''):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    avs = average_precision_score(y_true, y_pred)
    auc_score = auc(recall, precision)

    print('/n/n', name)

    plt.plot(recall, precision)
    plt.title("recall precision curve")

    plt.show()

    plt.hist(y_pred)
    plt.title("prediction histogram")
    plt.show()

    print(f"pr_auc = {auc_score}")

    print(f"average_precision_score = {avs}")

    print(f"holdout i = , roc_auc = {roc_auc_score(y_true, y_pred)}")



We will load one audio file and see the embeddings

In [3]:

df_preprocessed, features, target_feature = load_data()
df_preprocessed = df_preprocessed.dropna(subset=['target_binary_intrusion'], how='any')

X_train, X_test, y_train, y_test = train_test_split(df_preprocessed[features], df_preprocessed['target_binary_intrusion'],
                                      test_size=0.15, stratify=df_preprocessed['target_binary_intrusion'])
X_train, X_test = cv_preprocessing(X_train, X_test)


In [4]:
X_train.shape

(459, 85)

In [5]:

def embedding_function(x):
    
    weights = tf.Variable(tf.truncated_normal([42,1]))
    bias = tf.Variable(tf.truncated_normal([1]))
    
    a = (tf.nn.xw_plus_b(x,weights,bias))
    embeddings = tf.nn.relu(a)
    
    return embeddings

In [6]:
def relation_function(x):
    w1 = tf.Variable(tf.truncated_normal([2,3]))
    b1 = tf.Variable(tf.truncated_normal([3]))
    
    w2 = tf.Variable(tf.truncated_normal([3,5]))
    b2 = tf.Variable(tf.truncated_normal([5]))
    
    w3 = tf.Variable(tf.truncated_normal([5,1]))
    b3 = tf.Variable(tf.truncated_normal([1]))
    
    #layer1
    z1 = (tf.nn.xw_plus_b(x,w1,b1))
    a1 = tf.nn.relu(z1)
    
    #layer2
    z2 = tf.nn.xw_plus_b(a1,w2,b2)
    a2 = tf.nn.relu(z2)
    
    #layer3
    z3 = tf.nn.xw_plus_b(z2,w3,b3)

    #output
    y = tf.nn.sigmoid(z3)
    
    return y

In [7]:
xi = tf.placeholder(tf.float32, [None, 42])
xj = tf.placeholder(tf.float32, [None, 42])
y = tf.placeholder(tf.float32, [None, 1])


In [8]:
f_xi = embedding_function(xi)
f_xj = embedding_function(xj)
Z = tf.concat([f_xi,f_xj],axis=1)


In [9]:
d1 = X_train.iloc[:,:42].astype('float32').values
d2 = X_train.iloc[:,42:84].astype('float32').values
y_train = y_train.values.reshape(-1, 1)


In [10]:
y_train.shape

(459, 1)

In [11]:
relation_scores = relation_function(Z)
loss_function = tf.reduce_mean(tf.squared_difference(relation_scores,y))
optimizer = tf.train.AdamOptimizer(0.1)
train = optimizer.minimize(loss_function)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for episode in range(201):
    _, loss_value = sess.run([train, loss_function], 
                             feed_dict={xi:d1 + np.random.randn(*np.shape(d1))*0.05,
                                        xj:d2 + np.random.randn(*np.shape(d2))*0.05,
                                        y:y_train})
    if episode % 100 == 0:
        print("Episode {}: loss {:.3f} ".format(episode, loss_value))

Episode 0: loss 0.367 
Episode 100: loss 0.141 
Episode 200: loss 0.107 


In [ ]:
roc_auc_score(y_test, classification)

In [12]:
d1 = X_test.iloc[:,:42].astype('float32').values
d2 = X_test.iloc[:,42:84].astype('float32').values

feed_dict = {xi:d1,
             xj:d2}
classification = sess.run([relation_scores], feed_dict)
print(classification)

[array([[0.4134862],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [1.       ],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [1.       ],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [1.       ],
       [1.       ],
       [1.       ],
       [0.4134862],
       [1.       ],
       [0.4134862],
       [1.       ],
       [1.       ],
       [1.       ],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [0.4134862],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],
       [1.       ],
       [1.       ],
       [0.4134862],

Next, we split our data for training and testing with 75% training and 25% testing proportions:


Next, we set the epoch length to 13 and we use RMS prop for optimization.

In [13]:
cv = StratifiedKFold(5)

for train, test in cv.split(X, Y):
    
    x_train, y_train = X[train], Y[train]
    x_test, y_test = X[test], Y[test]
    epochs = 13
    rms = RMSprop()

    base_network = build_base_network(input_dim)
    audio_1 = x_train[:, 0]
    audio_2 = x_train[:, 1]
    feat_vecs_a = base_network(audio_a)
    feat_vecs_b = base_network(audio_b)
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])
    model = Model(inputs = [audio_a, audio_b], outputs=distance)
    model.compile(loss=contrastive_loss, optimizer=rms)



    model.fit([audio_1, audio_2], y_train.astype(float))

    print_results(y_test, model.predict([x_test[:,0], x_test[:,1]]))

NameError: name 'X' is not defined